In [5]:
import sqlite3
import os
from dotenv import load_dotenv
from langgraph.checkpoint.sqlite import SqliteSaver

# Load environment variables
load_dotenv(override=True)

# Set up database path
db_path = "memory.db"

# Create database connection
conn = sqlite3.connect(db_path, check_same_thread=False)

# Create SQLite checkpoint saver
sql_memory = SqliteSaver(conn)

print("✅ SQLite checkpoint memory initialized successfully!")
print(f"Database path: {os.path.abspath(db_path)}")
print(f"Connection status: {'Connected' if conn else 'Failed'}")

✅ SQLite checkpoint memory initialized successfully!
Database path: d:\Jeevan\Agentic_AI\LangGraph_Langchain\LangGraph\memory.db
Connection status: Connected


In [9]:
# Example: LangGraph with SQLite Memory Persistence
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import os

# Ensure SQLite memory is initialized (in case cells are run out of order)
if 'sql_memory' not in globals():
    print("🔄 Reinitializing SQLite memory...")
    db_path = "memory.db"
    conn = sqlite3.connect(db_path, check_same_thread=False)
    sql_memory = SqliteSaver(conn)
    print("✅ SQLite checkpoint memory reinitialized!")

# Check for OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("⚠️ No OpenAI API key found. Please set OPENAI_API_KEY in your .env file")
    print("This example will use a mock LLM for demonstration")
    
    class MockLLM:
        def invoke(self, messages):
            return AIMessage(content=f"Mock response to: {messages[-1].content}")
    
    llm = MockLLM()
else:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    print("✅ OpenAI LLM initialized")

# Define the state for our graph
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def call_model(state: AgentState):
    """Call the LLM with the current messages."""
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

def should_continue(state: AgentState):
    """Determine if we should continue the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Simple logic: continue if it's a human message
    if isinstance(last_message, HumanMessage):
        return "continue"
    else:
        return "end"

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)

# Add edges
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "agent",
        "end": END
    }
)

# Compile the graph with SQLite memory
app = workflow.compile(checkpointer=sql_memory)

print("✅ LangGraph workflow with SQLite memory created!")
print("Features:")
print("  - Persistent conversation memory")
print("  - State checkpointing")
print("  - Thread-based conversations")


✅ OpenAI LLM initialized
✅ LangGraph workflow with SQLite memory created!
Features:
  - Persistent conversation memory
  - State checkpointing
  - Thread-based conversations


In [10]:
# Test the persistent memory functionality
print("🧪 Testing SQLite Memory Persistence...")
print("=" * 50)

# Ensure all required variables are available
if 'app' not in globals():
    print("❌ LangGraph app not found. Please run the previous cell first.")
    print("This cell requires the LangGraph workflow to be compiled.")
    print("Please run the 'Example: LangGraph with SQLite Memory Persistence' cell first.")
else:
    print("✅ LangGraph app found, proceeding with tests...")
    
    # Test 1: Create a new conversation thread
    print("\n📝 Test 1: New conversation thread")
    thread_id = "test_conversation_1"
    config = {"configurable": {"thread_id": thread_id}}

    # Start a conversation
    result1 = app.invoke(
        {"messages": [HumanMessage(content="Hello! My name is Alice.")]},
        config=config
    )
    print(f"Response: {result1['messages'][-1].content}")

    # Continue the conversation
    result2 = app.invoke(
        {"messages": [HumanMessage(content="What's my name?")]},
        config=config
    )
    print(f"Response: {result2['messages'][-1].content}")

    # Test 2: Create another conversation thread
    print("\n📝 Test 2: Another conversation thread")
    thread_id_2 = "test_conversation_2"
    config2 = {"configurable": {"thread_id": thread_id_2}}

    result3 = app.invoke(
        {"messages": [HumanMessage(content="Hello! My name is Bob.")]},
        config=config2
    )
    print(f"Response: {result3['messages'][-1].content}")

    # Test 3: Go back to first thread (should remember Alice)
    print("\n📝 Test 3: Back to first thread (should remember Alice)")
    result4 = app.invoke(
        {"messages": [HumanMessage(content="What's my name again?")]},
        config=config
    )
    print(f"Response: {result4['messages'][-1].content}")

    print("\n✅ Memory persistence test completed!")
    print("Each thread maintains its own conversation history.")


🧪 Testing SQLite Memory Persistence...
✅ LangGraph app found, proceeding with tests...

📝 Test 1: New conversation thread
Response: Hello Alice! It's nice to meet you. How can I assist you today?
Response: Your name is Alice.

📝 Test 2: Another conversation thread
Response: Nice to meet you, Bob! How can I assist you today?

📝 Test 3: Back to first thread (should remember Alice)
Response: Your name is Alice.

✅ Memory persistence test completed!
Each thread maintains its own conversation history.


In [11]:
# Advanced Memory Features
print("🔧 Advanced SQLite Memory Features...")
print("=" * 50)

# Ensure all required variables are available
if 'app' not in globals() or 'conn' not in globals():
    print("❌ Required variables not found. Please run the previous cells first.")
    print("This cell requires the LangGraph app and database connection.")
    print("Please run the first two cells in order.")
else:
    print("✅ All required variables found, proceeding with advanced features...")
    
    # List all conversation threads
    print("\n📋 Available conversation threads:")
    try:
        # Get all thread IDs from the database
        cursor = conn.cursor()
        cursor.execute("SELECT DISTINCT thread_id FROM checkpoints")
        threads = cursor.fetchall()
        
        if threads:
            for i, (thread_id,) in enumerate(threads, 1):
                print(f"  {i}. {thread_id}")
        else:
            print("  No conversation threads found")
            
    except Exception as e:
        print(f"  Error retrieving threads: {e}")

    # Get conversation history for a specific thread
    if 'config' in globals():
        print(f"\n📜 Conversation history for thread: {config['configurable']['thread_id']}")
        try:
            # Get the latest state for the thread
            state = app.get_state(config)
            if state and state.values:
                messages = state.values.get("messages", [])
                print(f"  Total messages: {len(messages)}")
                for i, msg in enumerate(messages, 1):
                    msg_type = "Human" if isinstance(msg, HumanMessage) else "AI"
                    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
                    print(f"    {i}. [{msg_type}]: {content}")
            else:
                print("  No conversation history found")
                
        except Exception as e:
            print(f"  Error retrieving history: {e}")
    else:
        print("\n📜 No conversation thread selected for history display")

    # Database statistics
    print(f"\n📊 Database Statistics:")
    try:
        cursor.execute("SELECT COUNT(*) FROM checkpoints")
        checkpoint_count = cursor.fetchone()[0]
        print(f"  Total checkpoints: {checkpoint_count}")
        
        cursor.execute("SELECT COUNT(DISTINCT thread_id) FROM checkpoints")
        thread_count = cursor.fetchone()[0]
        print(f"  Total threads: {thread_count}")
        
        # Get database file size
        db_size = os.path.getsize(db_path)
        print(f"  Database size: {db_size} bytes ({db_size/1024:.2f} KB)")
        
    except Exception as e:
        print(f"  Error getting statistics: {e}")

    print("\n✅ Advanced memory features demonstrated!")


🔧 Advanced SQLite Memory Features...
✅ All required variables found, proceeding with advanced features...

📋 Available conversation threads:
  1. test_conversation_1
  2. test_conversation_2

📜 Conversation history for thread: test_conversation_1
  Total messages: 12
    1. [Human]: Hello! My name is Alice.
    2. [AI]: Hello Alice! It's nice to meet you. How are you today?
    3. [Human]: What's my name?
    4. [AI]: Your name is Alice.
    5. [Human]: What's my name again?
    6. [AI]: Your name is Alice.
    7. [Human]: Hello! My name is Alice.
    8. [AI]: Hello Alice! It's nice to meet you. How can I assist you today?
    9. [Human]: What's my name?
    10. [AI]: Your name is Alice.
    11. [Human]: What's my name again?
    12. [AI]: Your name is Alice.

📊 Database Statistics:
  Total checkpoints: 24
  Total threads: 2
  Database size: 4096 bytes (4.00 KB)

✅ Advanced memory features demonstrated!


# LangGraph SQLite Memory Persistence

This notebook demonstrates how to use SQLite as a persistent memory store for LangGraph applications.

## What is SQLite Memory in LangGraph?

SQLite memory allows LangGraph applications to:
- **Persist conversation state** across sessions
- **Maintain multiple conversation threads** independently
- **Store checkpoints** for complex workflows
- **Resume conversations** after application restarts

## Key Features Demonstrated:

### ✅ **Persistent State Management**
- Conversation history is saved to SQLite database
- State persists across application restarts
- Multiple conversation threads supported

### ✅ **Thread-Based Conversations**
- Each conversation has a unique thread ID
- Threads maintain independent conversation histories
- Easy to switch between different conversations

### ✅ **Database Operations**
- View all conversation threads
- Retrieve conversation history
- Monitor database statistics
- Manage checkpoint data

## Setup Requirements:

### Required:
- **langgraph-checkpoint**: `pip install langgraph-checkpoint`
- **sqlite3**: Built into Python standard library

### Optional:
- **OpenAI API Key**: For real LLM responses (falls back to mock if not available)

## Database Schema:

The SQLite database automatically creates tables to store:
- **checkpoints**: Main table storing conversation states
- **thread_id**: Unique identifier for each conversation
- **checkpoint_id**: Sequential ID for each state checkpoint
- **metadata**: Additional information about each checkpoint

## Use Cases:

1. **Chat Applications**: Maintain conversation context across sessions
2. **Multi-User Systems**: Separate conversations for different users
3. **Complex Workflows**: Save intermediate states for long-running processes
4. **Debugging**: Inspect conversation history and state transitions
5. **Analytics**: Analyze conversation patterns and user interactions

## Next Steps:

1. Explore more complex state management patterns
2. Implement user authentication with thread management
3. Add conversation analytics and insights
4. Build web interfaces with persistent memory
5. Scale to production with proper database management


In [12]:
# Complete Setup Cell - Run this if you get NameError
print("🔧 Complete SQLite Memory Setup")
print("=" * 40)

# Import all required modules
import sqlite3
import os
from typing import Annotated, TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv(override=True)

# Set up database path
db_path = "memory.db"

# Create database connection
conn = sqlite3.connect(db_path, check_same_thread=False)

# Create SQLite checkpoint saver
sql_memory = SqliteSaver(conn)

print("✅ SQLite checkpoint memory initialized successfully!")
print(f"Database path: {os.path.abspath(db_path)}")
print(f"Connection status: {'Connected' if conn else 'Failed'}")

# Check for OpenAI API key and create LLM
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("⚠️ No OpenAI API key found. Using mock LLM for demonstration")
    
    class MockLLM:
        def invoke(self, messages):
            return AIMessage(content=f"Mock response to: {messages[-1].content}")
    
    llm = MockLLM()
else:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    print("✅ OpenAI LLM initialized")

# Define the state for our graph
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def call_model(state: AgentState):
    """Call the LLM with the current messages."""
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

def should_continue(state: AgentState):
    """Determine if we should continue the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Simple logic: continue if it's a human message
    if isinstance(last_message, HumanMessage):
        return "continue"
    else:
        return "end"

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)

# Add edges
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "agent",
        "end": END
    }
)

# Compile the graph with SQLite memory
app = workflow.compile(checkpointer=sql_memory)

print("✅ LangGraph workflow with SQLite memory created!")
print("Features:")
print("  - Persistent conversation memory")
print("  - State checkpointing")
print("  - Thread-based conversations")
print("\n🎯 All variables are now available:")
print("  - sql_memory: SQLite checkpoint saver")
print("  - app: Compiled LangGraph workflow")
print("  - conn: Database connection")
print("  - llm: Language model")
print("\n✅ Setup complete! You can now run the test cells.")


🔧 Complete SQLite Memory Setup
✅ SQLite checkpoint memory initialized successfully!
Database path: d:\Jeevan\Agentic_AI\LangGraph_Langchain\LangGraph\memory.db
Connection status: Connected
✅ OpenAI LLM initialized
✅ LangGraph workflow with SQLite memory created!
Features:
  - Persistent conversation memory
  - State checkpointing
  - Thread-based conversations

🎯 All variables are now available:
  - sql_memory: SQLite checkpoint saver
  - app: Compiled LangGraph workflow
  - conn: Database connection
  - llm: Language model

✅ Setup complete! You can now run the test cells.
